# **Quantum-inspired Generative models**

With the recent rise in interest in generative models in the classical machine learning domain, there have also been advancements along the lines of using generative models with quantum computing. This interest has varied from using quantum Generative Adversarial Networks (qGANs) to efficiently load data onto a quantum computer [3] to applications in finance [4].  

The objective of this project is to explore the prospects of generative models in the quantum computing paradigm. The paper [1] leverages constrained optimisation problems to define metrics to gauge the models generalization capabilities. The objective function in these problems being similar to the use-case of modern portfolio theory given in [2], is used to demonstrate this advantage. The paper also suggests that quantum-inspired generative models can offer upto a 68× enhancement in the generation of synthetic data compared to classical Generative Adversarial Networks (GANs). 



---



# What are Quantum Generative Adversarial Networks (qGANs)?

The loading of data into quantum states is an obstacle to implementing algorithms on NISQ-era devices because it adds to the computational complexity of the overall algorithm. And in doing so, impairs the quantum advantage that is hoped for in the computation. Quantum generative adversarial networks [3] were proposed to provide some relief to the complexity overhead. 

This efficient and approximate quantum state loading is done by the algorithm learning the implicit probability distribution underlying the data samples, and then loading this probability distribution into quantum states using a variational circuit. This brings the complexity of loading data from $O(2^n)$ to $O(poly(n))$.

Classical Generative Adversarial Networks (GANs) consist of two parts - the *generator* and the *discriminator*. The generator generates fake data samples with the goal of making them as indistinguishable from real data samples. While the discriminator's goal is to figure out if a given sample is real or fake. Both components are working against each other (adversarially) and their competition captures the implicit probability distribution more clearly. 

A *qGAN* is when either of the two components of this algorithm is done on a quantum computer. In this instance [3], the qGAN refers to a quantum generator that can create a quantum state which loyally represents the data’s underlying probability distribution.





---



# Portfolio optimisation
Portfolio optimisation refers to the allocation of resources across various asset classes to acheive optimal returns with respect to the risk entailed with each investment option i.e. maximise returns while minimizing risk.

The modern portfolio theory (MPT) is a practical method for selecting investments in order to maximize their overall returns within an acceptable level of risk, a key feature of which is diversification. As such, it is described by an objective function with various parameters to be optimised, given a dataset from which the relationship between variables must be extracted. 

Portfolio optimisation is a ripe problem for quantum computing to provide an advantage for as classical computing has reached its limitations. It would benefit a great deal from quantum computers in terms of accuracy, even if efficiency may have to wait until further improvements in quantum hardware. 

The paper [5] gives promising results by framing MPT solution as a solution for a variational quantum eigensolver, where the Hamiltonian is replaced with the objective function relevant to the problem. The paper goes on to evaluate these results on various devices, including D-Wave's quantum annealer which provide the most constructive results.



---



# Evaluation of Generalizing capabilities
The paper [1] aims to provide a well-defined framework with metrics to evaluate the capabilities of various distribution learning methods, both classical and quantum. They acheive this by building robust sample-based metrics to gauge the performance of different methods with consideration to the unique pitfalls of each method. 





---



# Data Pre-processing
As stated in [1], portfolio optimisation requires a decision regarding asset allocation, whether to allocate resources to it or not. One way of encoded the data is by having every stock represented by a single qubit, the outcome of which will either be a 0 or 1, to indicate the decision. This instance will use 6 qubits and so the label for each row given is in the form of a bitstring consisting of 6 bits each, indicating which of the 6 assets must be allocated resources at a given time. The value of the bit is evaluated based on the value of the stock in the next progression.

In [ ]:
pip install Nasdaq-Data-Link

I have obtained the dataset for this project using the online NASDAQ data link and will attach a file consisting of the processed dataset alongwith the project.

In [73]:
import nasdaqdatalink

nasdaqdatalink.ApiConfig.api_key = api_key

MSFT_data = nasdaqdatalink.get("WIKI/MSFT", rows = 3941)['Close']
XOM_data = nasdaqdatalink.get("WIKI/XOM", rows = 3941)['Close']
GOLD_data = nasdaqdatalink.get("WIKI/GOLD", rows = 3941)['Close']
BAC_data = nasdaqdatalink.get("WIKI/BAC", rows = 3941)['Close']
DE_AGRI_data = nasdaqdatalink.get("WIKI/DE", rows = 3941)['Close']
BA_data = nasdaqdatalink.get("WIKI/BA", rows = 3941)['Close']


In [72]:
target = []
for i in range(len(MSFT_data)-1):
  bit_string = ''

  if MSFT_data[i+1] - MSFT_data[i] > 0:
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  if ((XOM_data[i+1] - XOM_data[i]) > 0):
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  if ((GOLD_data[i+1] - GOLD_data[i]) > 0):
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  if ((BAC_data[i+1] - BAC_data[i]) > 0):
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  if ((DE_AGRI_data[i+1] - DE_AGRI_data[i]) > 0):
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  if ((BA_data[i+1] - BA_data[i]) > 0):
    bit_string = bit_string + '1'
  else:
    bit_string = bit_string + '0'

  target.append(bit_string)

df = pd.DataFrame(MSFT_data)
df.insert(1,'XOM', XOM_data, True)
df.insert(2,'GOLD', GOLD_data, True)
df.insert(3,'BAC', BAC_data, True)
df.insert(4,'DE_AGRI', DE_AGRI_data, True)
df.insert(5,'BA', BA_data, True)
df.drop(df.tail(1).index,inplace=True)
df.insert(6,'Labels', target, True)
print(df)
df.to_csv("Nasdaq_data.csv")

            Close    XOM   GOLD    BAC  DE_AGRI      BA  Labels
Date                                                           
2002-07-31  47.98  36.76  12.80  66.50    42.02   41.52  001000
2002-08-01  45.75  33.65  12.90  64.80    41.65   40.63  001000
2002-08-02  44.41  33.61  13.00  63.51    39.95   39.85  001000
2002-08-05  43.99  32.50  13.05  62.06    39.81   38.61  111111
2002-08-06  45.67  33.40  13.08  63.73    40.90   39.41  111111
...           ...    ...    ...    ...      ...     ...     ...
2018-03-20  93.13  73.99  82.26  31.98   159.99  337.63  011000
2018-03-21  92.48  75.04  84.00  31.87   158.05  337.10  000000
2018-03-22  89.79  73.50  80.52  30.55   151.58  319.61  001001
2018-03-23  87.18  72.89  83.92  29.17   148.00  321.00  111111
2018-03-26  93.78  74.00  85.22  30.44   153.34  328.97  000000

[3940 rows x 7 columns]


# Further Work
The given data encoding can be used to implement qGANs given in [4]. There are also other prospective uses for qGANs in terms of generating synthetic data. 

Other distribution learning methods such as Quantum Boltzmann Machine and Quantum Circuit Born Machine (QCBM) have shown to compete very well with classical methods [6]. Another interesting facet to these is the performance of the generative model being correlated to the entanglement capacity of the system, which emphasises the integral role the quantum circuit plays in bringing about this advantage. 


# References
1. K. Gili, M. Mauri, A. Perdomo-Ortiz, *Evaluating Generalization in Classical and Quantum Generative Models* (26 May 2022), arXiv:2201.08770v2.
2. Multiverse Computing: Optimizing Financial Portfolios with Quantum Computing - https://www.dwavesys.com/media/5qahck2o/multiverse_case_study_v8.pdf
3. Zoufal, C., Lucchi, A. & Woerner, S. Quantum Generative Adversarial Networks for learning and loading random distributions. npj Quantum Inf 5, 103 (2019). https://doi.org/10.1038/s41534-019-0223-2
4. https://pennylane.ai/qml/demos/tutorial_quantum_gans.html
5. S. Mugel, C. Kuchkovsky, E. Sanchez, S. Fernandez-Lorenzo, J. Luis-Hita, E. Lizaso, and R. Orus, '*Dynamic Portfolio Optimization with Real Datasets Using Quantum Processors and Quantum-Inspired Tensor Networks*' (2021), arXiv:2007.00017v2.
6. B. Coyle, M. Henderson, J. Chan Jin Le, N. Kumar, M. Paini, E. Kashefi, *Quantum versus Classical Generative Modelling in Finance* (2020), arXiv:2008.00691v1.